In [ ]:
# TODO: per cell!

import h5py as h
import numpy as np
import pandas as pd

import skimage.morphology as morph
import skimage.measure as measure

import os
import skan

def lengths_3d(array):

    # skeletonize image
    skel = morph.skeletonize(array, method='lee')
    #fat_skel = morph.binary_dilation(skel)

    # label skeleton per branch
    labs2 = skan.Skeleton(skel).path_label_image()
    #fat_labs = skan.Skeleton(fat_skel).path_label_image()

    # get data of the skeletons, spacing = dimensions of the image.
    branch_data = skan.summarize(skan.Skeleton(skel,spacing=(0.2,0.045,0.045)))

    # make df
    # calculate average branch length per object, max branch length and total length
    av_branch = branch_data.groupby('skeleton-id').mean()['branch-distance'].tolist()
    max_branch = branch_data.groupby('skeleton-id').max()['branch-distance'].tolist()
    n_branch = branch_data.groupby('skeleton-id').count()['branch-distance'].tolist()
    total_length = branch_data.groupby('skeleton-id').sum()['branch-distance'].tolist()

    return pd.DataFrame({'av_branch_length':av_branch,'max_branch_length':max_branch,'n_branches':n_branch,'total_mito_length':total_length})

# folder with segmentations from Ilastik
dir = r"X:\Kleele\Nov_2024\02-Analysis\241108_RPE1_drug_treatments_screen\Ilastik_segmentation"

# set threshold for smallest mitochondria possible
small_mito = 100

imgs = os.listdir(dir)

res = []
for file in os.listdir(dir):
    if file.endswith('tation.h5'):
        basename = file.split('_decon')[0]
        
        array = h.File(os.path.join(dir, file))['exported_data'][:,:,:,0]

        array[array == 2] = 0

        # remove small particles
        small_rem = morph.remove_small_objects(morph.label(array),min_size=small_mito)
        array = np.array(small_rem > 0) * 1

        temp = lengths_3d(array)
        temp['image'] = basename
        res.append(temp)

res = pd.concat(res)
res.to_csv(os.path.join(dir, 'mito_lengths.csv'), index=False)


In [ ]:
# check segmentation with napari. 
import napari 



In [ ]:
# area / form factor calculation 

def calculate_area_form(array, dimensions=(0.2, 0.045, 0.045), small_mito=100):
    props = ['area','eccentricity','axis_major_length', 'axis_minor_length', 'bbox', 'centroid']
    # label array and remove small particles
    small_rem = morph.remove_small_objects(morph.label(array),min_size=small_mito)
    array = np.array(small_rem > 0) * 1

    # label
    lab = measure.label(array)

    # calculate regionprops
    df = pd.DataFrame(measure.regionprops_table(lab, properties=props))

    # convert to right dimensions
    df['area'] = df['area'] * dimensions[0] * dimensions[1] * dimensions[2]
    df['axis_major_length'] = df['axis_major_length'] * dimensions[1]
    df['axis_minor_length'] = df['axis_minor_length'] * dimensions[1]



